# Title
**Author**:  Greg Slater <br>
**Date**:  27th September 2024 <br>
**Dataset Scope**: <br>
**Report Type**: Ad-hoc analysis <br>

## Purpose
Just some quick tests of simplifying some of the very large polygons in `flood-risk-zone` dataset to help with platform API query optimisation

In [ ]:
import pandas as pd
import geopandas as gpd
import shapely
import urllib

pd.set_option("display.max_rows", 100)

# data_dir = "../../data/db_downloads/"
# os.makedirs(data_dir, exist_ok=True)

In [ ]:
def datasette_query(db, sql_string):
    params = urllib.parse.urlencode({
        "sql": sql_string,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/{db}.csv?{params}"
    df = pd.read_csv(url)
    return df

## Data Import

In [ ]:
# get geometry facts for entity 65123947 from datasette
frz_polys = datasette_query("flood-risk-zone", """                       
    select value
    from fact
    where entity = 65123947 and field = "geometry"
    """)

# turn into gdf
frz_polys['value'] = frz_polys['value'].apply(shapely.wkt.loads)
frz_polys = gpd.GeoDataFrame(frz_polys, geometry='value')
frz_polys.set_crs(4326, inplace=True)


In [ ]:
# gdf for the example polygon in the slow API call
user_poly = pd.DataFrame({"wkt":["MULTIPOLYGON (((-2.241761 51.864916,-2.241601 51.864828,-2.241561 51.864854,-2.241617 51.864886,-2.2416 51.864897,-2.241703 51.864957,-2.241761 51.864916)))"]})

user_poly['wkt'] = user_poly['wkt'].apply(shapely.wkt.loads)
user_poly = gpd.GeoDataFrame(user_poly, geometry='wkt')
user_poly.set_crs(4326, inplace=True)

user_poly.explore()

## Analysis

In [ ]:
# map big frz geom with user poly
m = frz_polys.iloc[[1]].explore()

user_poly.explore(
    m = m,
    color = "red"
)

In [ ]:
# same with simplification of frz poly
m = frz_polys.iloc[[1]].simplify(.0001).explore()

user_poly.explore(
    m = m,
    color = "red"
)